# Mate detection

## Basic Setup

In [4]:
# Clone mate-detection repo
!git clone https://github.com/75-70-Robots/mate-detection.git

Clonando en 'mate-detection'...
remote: Enumerating objects: 945, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 945 (delta 133), reused 150 (delta 133), pack-reused 795
Recibiendo objetos: 100% (945/945), 125.87 MiB | 7.09 MiB/s, listo.
Resolviendo deltas: 100% (522/522), listo.


In [5]:
# Clone yolov5 repo
!git clone https://github.com/ultralytics/yolov5

Clonando en 'yolov5'...
remote: Enumerating objects: 6814, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 6814 (delta 71), reused 104 (delta 64), pack-reused 6696
Recibiendo objetos: 100% (6814/6814), 8.70 MiB | 4.81 MiB/s, listo.
Resolviendo deltas: 100% (4676/4676), listo.


In [6]:
!pip3 install -r yolov5/requirements.txt

In [7]:
!pip3 install -r requirements.txt

In [8]:
import os
import cv2
import copy
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import shutil
import yaml

## Data parsing

In [19]:
IMG_DIR = "mate-detection/images/mates/"
XML_DIR = "mate-detection/labels/mates/"
labels = ["mate"]
tamanio = 416

def leer_mate_labels(ann_dir, img_dir, labels=[]):
    all_imgs = []
    for ann in sorted(os.listdir(ann_dir)):
        img = {'object':[]}
        tree = ET.parse(ann_dir + ann)
        
        for elem in tree.iter():
            if 'filename' in elem.tag:
                img['filename'] = img_dir + elem.text
            if 'width' in elem.tag:
                img['width'] = int(elem.text)
            if 'height' in elem.tag:
                img['height'] = int(elem.text)
            if 'object' in elem.tag or 'part' in elem.tag:
                obj = {}
                
                for attr in list(elem):
                    if 'name' in attr.tag:
                        obj['name'] = attr.text.lower()
                        if len(labels) > 0 and obj['name'] not in labels:
                            break
                        else:
                            img['object'] += [obj]
                            
                    if 'bndbox' in attr.tag:
                        for dim in list(attr):
                            if 'xmin' in dim.tag:
                                obj['xmin'] = int(round(float(dim.text)))
                            if 'ymin' in dim.tag:
                                obj['ymin'] = int(round(float(dim.text)))
                            if 'xmax' in dim.tag:
                                obj['xmax'] = int(round(float(dim.text)))
                            if 'ymax' in dim.tag:
                                obj['ymax'] = int(round(float(dim.text)))

        if len(img['object']) > 0:
            all_imgs += [img]
                        
    return all_imgs

train_imgs = leer_mate_labels(XML_DIR, IMG_DIR, labels)
print(len(train_imgs))
print(train_imgs)

361
[{'object': [{'name': 'mate', 'xmin': 246, 'ymin': 338, 'xmax': 717, 'ymax': 951}], 'filename': 'mate-detection/images/mates/Manes-01.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 192, 'ymin': 450, 'xmax': 628, 'ymax': 887}], 'filename': 'mate-detection/images/mates/Manes-02.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 280, 'ymin': 515, 'xmax': 733, 'ymax': 1114}], 'filename': 'mate-detection/images/mates/Manes-03.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 200, 'ymin': 483, 'xmax': 540, 'ymax': 955}], 'filename': 'mate-detection/images/mates/Manes-04.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 86, 'ymin': 410, 'xmax': 586, 'ymax': 875}], 'filename': 'mate-detection/images/mates/Manes-05.jpeg', 'width': 958, 'height': 1280}, {'object': [{'name': 'mate', 'xmin': 132, 'ymin': 557, 'xmax': 539, 'ymax': 991}], 'filename': 'mate-detection/images/mates/Manes-06.jpeg', 

In [20]:
def create_labels_id(labels):
    labels_id = {}
    id_count = 0
    for label in labels:
        labels_id[label] = id_count
        id_count += 1
    return labels_id

labels_id = create_labels_id(labels)
labels_id

{'mate': 0}

In [21]:
train_valid_split = int(0.8*len(train_imgs))
np.random.shuffle(train_imgs)
validation_imgs = train_imgs[train_valid_split:]
training_imgs = train_imgs[:train_valid_split]
print('train:',len(training_imgs), 'validate:',len(validation_imgs))

train: 288 validate: 73


In [22]:
def transform_dict_in_dataframe(img_dict):
    df = pd.DataFrame()
    for image in img_dict:
        for obj in image['object']:
            x1 = obj['xmin']
            x2 = obj['xmax']
            y1 = obj['ymin']
            y2 = obj['ymax']
            w = image['width']
            h = height = image['height']
            width = (x2-x1)/w
            height = (y2-y1)/h
            xcenter = (x1+(x2-x1)/2)/w
            ycenter = (y1+(y2-y1)/2)/h
            label = obj['name']
            filename = image['filename']
            new_row = {'filename': filename, 'label': label, 'xcenter': xcenter, 'ycenter': ycenter, 'width': width,
                      'height': height, 'image_height':h, 'image_width':w}
            df = df.append(new_row, ignore_index=True)

    return df

In [23]:
def create_yolo_dataset(train_dataframe, labels_id, trained_img_path, trained_lbl_path):
    
    if not os.path.exists(trained_img_path):
        os.makedirs(trained_img_path)
    if not os.path.exists(trained_lbl_path):
        os.makedirs(trained_lbl_path)

    img_names = list(train_dataframe['filename'].unique())

    for full_name in img_names:
        name = full_name.split('/')[-1]
        if not os.path.exists(trained_img_path+name):
            shutil.copy(IMG_DIR+name,trained_img_path+name)
        img = cv2.imread(IMG_DIR+name)
        h, w = img.shape[:2]
        f_name = name.split('.')[0]

        with open(trained_lbl_path+f_name+".txt",'w') as label:
            ldf=train_dataframe.loc[train_dataframe['filename']==full_name]
            for index, row in ldf.iterrows():
                l = labels_id[row['label']]
                xcenter = row['xcenter']
                ycenter = row['ycenter']
                width = row['width']
                height = row['height']
                l_data = f"{l} {xcenter} {ycenter} {width} {height}\n"
                label.write(l_data)

In [24]:
def create_yolo_yaml_configuration(training_img_path, validation_img_path, labels, yaml_name, yolo_model='s'):
    yaml_data = {
      'train': training_img_path,
      'val': validation_img_path,
      'nc':len(labels),
      'names':labels
    }

    with open(yaml_name,'w') as f:
        yaml.dump(yaml_data, f)

    with open(f'yolov5/models/yolov5{yolo_model}.yaml') as f:
        ydata = yaml.load(f, Loader=yaml.FullLoader) 

    ydata['nc'] = len(labels)
    with open('yolov5_custom.yaml','w') as cy:
        yaml.dump(ydata,cy)

In [25]:
TRAINED_IMG_PATH = 'data/images/'
TRAINED_LBL_PATH = 'data/labels/'
train_df = transform_dict_in_dataframe(training_imgs)
create_yolo_dataset(train_df, labels_id, TRAINED_IMG_PATH, TRAINED_LBL_PATH)
train_df

,filename,height,image_height,image_width,label,width,xcenter,ycenter
0,mate-detection/images/mates/mamarkosich-17.jpeg,0.640764,785.0,828.0,mate,0.434783,0.461353,0.459236
1,mate-detection/images/mates/fmazzoni-12.jpeg,0.144380,1032.0,774.0,mate,0.197674,0.398579,0.471415
2,mate-detection/images/mates/mamarkosich-03.jpeg,0.794403,929.0,828.0,mate,0.550725,0.533816,0.432723
3,mate-detection/images/mates/apayaslian-20.jpg,0.632870,2160.0,2160.0,mate,0.482407,0.581481,0.372454
4,mate-detection/images/mates/lolguin-38.jpeg,0.424490,245.0,206.0,mate,0.349515,0.218447,0.734694
...,...,...,...,...,...,...,...,...
284,mate-detection/images/mates/apayaslian-33.jpg,0.422685,2160.0,2160.0,mate,0.339352,0.504861,0.429861
285,mate-detection/images/mates/lolguin-05.jpeg,0.147656,1280.0,720.0,mate,0.198611,0.492361,0.521484
286,mate-detection/images/mates/aaackermann-26.jpg,0.539844,1280.0,958.0,mate,0.377871,0.811065,0.575391
287,mate-detection/images/mates/aaackermann-13.jpg,0.414844,1280.0,958.0,mate,0.211900,0.460856,0.762109


In [26]:
VALIDATION_IMG_PATH = 'valid/images/'
VALIDATION_LBL_PATH = 'valid/labels/'
valid_df = transform_dict_in_dataframe(validation_imgs)
create_yolo_dataset(valid_df, labels_id, VALIDATION_IMG_PATH, VALIDATION_LBL_PATH)
valid_df

,filename,height,image_height,image_width,label,width,xcenter,ycenter
0,mate-detection/images/mates/aaackermann-15.jpg,0.482031,1280.0,958.0,mate,0.330898,0.166493,0.758984
1,mate-detection/images/mates/fmazzoni-01.jpeg,0.090116,1032.0,774.0,mate,0.124031,0.459948,0.563469
2,mate-detection/images/mates/mamarkosich-18.jpeg,0.441935,930.0,828.0,mate,0.338164,0.306763,0.686559
3,mate-detection/images/mates/mamarkosich-26.jpeg,0.952756,762.0,828.0,mate,0.497585,0.347826,0.523622
4,mate-detection/images/mates/cflorez-22.jpeg,0.405469,1280.0,720.0,mate,0.623611,0.575694,0.743359
...,...,...,...,...,...,...,...,...
68,mate-detection/images/mates/fmazzoni-19.jpeg,0.370801,774.0,1032.0,mate,0.169574,0.658430,0.451550
69,mate-detection/images/mates/npfernandeztheille...,0.524444,225.0,192.0,mate,0.494792,0.544271,0.737778
70,mate-detection/images/mates/npfernandeztheille...,0.363889,360.0,640.0,mate,0.232813,0.399219,0.787500
71,mate-detection/images/mates/npfernandeztheille...,0.445000,1000.0,750.0,mate,0.462667,0.483333,0.590500


In [27]:
YAML_NAME = 'mate.yaml'
yolo_model = 'x'
create_yolo_yaml_configuration(TRAINED_IMG_PATH,VALIDATION_IMG_PATH,labels,YAML_NAME,yolo_model)

## Train

In [28]:
!python3 yolov5/train.py --img 640 --batch 1 --epochs 150 --data mate.yaml --weights yolov5x.pt --name yolov5_mates

github: skipping check (not a git repository)
YOLOv5 🚀 v5.0-130-gfdbe527 torch 1.8.1+cu102 CUDA:0 (GeForce GTX 1050, 4042.375MB)

Namespace(adam=False, artifact_alias='latest', batch_size=1, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='mate.yaml', device='', entity=None, epochs=150, evolve=False, exist_ok=False, global_rank=-1, hyp='./yolov5/data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov5_mates', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5_mates33', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=1, upload_dataset=False, weights='yolov5x.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-06-02 10:18:39.493200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not loa

## Material recognition

In [150]:
material_labels = ['madera', 'plastico', 'calabaza', 'metal']
material_imgs = leer_mate_labels(XML_DIR, IMG_DIR, material_labels)
len(train_imgs)

347

In [151]:
material_df = transform_dict_in_dataframe(material_imgs)
material_df

,filename,height,image_height,image_width,label,width,xcenter,ycenter
0,mate-detection/images/mates/Manes-01.jpeg,0.478906,1280.0,958.0,madera,0.491649,0.504697,0.505078
1,mate-detection/images/mates/Manes-02.jpeg,0.341406,1280.0,958.0,plastico,0.455115,0.430063,0.523828
2,mate-detection/images/mates/Manes-03.jpeg,0.467969,1280.0,958.0,madera,0.472860,0.530793,0.637891
3,mate-detection/images/mates/Manes-04.jpeg,0.368750,1280.0,958.0,madera,0.354906,0.388309,0.563281
4,mate-detection/images/mates/Manes-05.jpeg,0.363281,1280.0,958.0,plastico,0.521921,0.348643,0.500391
...,...,...,...,...,...,...,...,...
342,mate-detection/images/mates/npfernandeztheille...,0.423333,600.0,480.0,madera,0.412500,0.277083,0.785000
343,mate-detection/images/mates/npfernandeztheille...,0.509124,548.0,822.0,calabaza,0.282238,0.324818,0.694343
344,mate-detection/images/mates/npfernandeztheille...,0.790909,220.0,449.0,calabaza,0.253898,0.726058,0.531818
345,mate-detection/images/mates/npfernandeztheille...,0.534375,640.0,470.0,calabaza,0.551064,0.375532,0.690625


In [153]:
MATERIAL_TRAINED_IMG_PATH = 'material_data/images/'
MATERIAL_TRAINED_LBL_PATH = 'material_data/labels/'
YAML_NAME = 'material.yaml'
material_labels_id = create_labels_id(material_labels)
create_yolo_dataset(material_df, material_labels_id, MATERIAL_TRAINED_IMG_PATH, MATERIAL_TRAINED_LBL_PATH)
create_yolo_yaml_configuration(MATERIAL_TRAINED_IMG_PATH,material_labels, YAML_NAME)

In [1]:
!python3 yolov5/train.py --img 640 --batch 1 --epochs 30 --data material.yaml --weights yolov5s.pt --name yolov5_materials

github: skipping check (not a git repository)
YOLOv5 🚀 v5.0-129-gd833ab3 torch 1.8.1+cu102 CUDA:0 (GeForce GTX 1050, 4042.375MB)

Namespace(adam=False, artifact_alias='latest', batch_size=1, bbox_interval=-1, bucket='', cache_images=False, cfg='', data='material.yaml', device='', entity=None, epochs=30, evolve=False, exist_ok=False, global_rank=-1, hyp='./yolov5/data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov5_materials', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov5_materials6', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=1, upload_dataset=False, weights='yolov5s.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-06-01 22:31:25.282144: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Cou


     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
      7/29    0.856G   0.03279   0.01803   0.02743   0.07825         2       640
               Class      Images      Labels           P           R      mAP@.5
                 all         347         347       0.201       0.535       0.216       0.112

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
      8/29    0.856G   0.03107   0.01667   0.02768   0.07542         2       640
               Class      Images      Labels           P           R      mAP@.5
                 all         347         347       0.212       0.428       0.215       0.102

     Epoch   gpu_mem       box       obj       cls     total    labels  img_size
      9/29    0.856G   0.02956   0.01637   0.02665   0.07259         1       640
               Class      Images      Labels           P           R      mAP@.5
                 all         347         347       0.232       0.637       0.272  

Optimizer stripped from runs/train/yolov5_materials6/weights/best.pt, 14.4MB


In [ ]:
!python3 yolov5/detect.py --source test_images/ --weights runs/train/yolov5_mates28/weights/best.pt --conf 0.75

## Detection

### Image

In [3]:
!python3 yolov5/detect.py --source test_images/ --weights runs/train/yolov5_mates32/weights/best.pt --conf 0.5

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.5, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='test_images/', update=False, view_img=False, weights=['runs/train/yolov5_mates32/weights/best.pt'])
YOLOv5 🚀 v5.0-129-gd833ab3 torch 1.8.1+cu102 CUDA:0 (GeForce GTX 1050, 4042.375MB)

Fusing layers... 
Model Summary: 476 layers, 87198694 parameters, 0 gradients, 217.1 GFLOPS
image 1/13 /home/payas/Desktop/Facultad/Robots/TP1/test_images/mate1.jpeg: 640x640 Done. (0.228s)
image 2/13 /home/payas/Desktop/Facultad/Robots/TP1/test_images/mate10.jpg: 640x640 Done. (0.224s)
image 3/13 /home/payas/Desktop/Facultad/Robots/TP1/test_images/mate11.jpeg: 352x640 3 mates, Done. (0.133s)
image 4/13 /home/payas/Desktop/Facultad/Robots/TP1/test_images/mate12.jpeg: 640x640 Done. (0.224s)
image 5/13 /ho

### Video

### Webcam

In [9]:
!python3 yolov5/detect.py --source 0 --weights runs/train/yolov5_mates28/weights/best.pt --img 640 --conf 0.75

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.75, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=640, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='0', update=False, view_img=False, weights=['runs/train/yolov5_mates28/weights/best.pt'])
YOLOv5 🚀 v5.0-129-gd833ab3 torch 1.8.1+cu102 CUDA:0 (GeForce GTX 1050, 4042.375MB)

Fusing layers... 
Model Summary: 476 layers, 87198694 parameters, 0 gradients, 217.1 GFLOPS
1/1: 0...  success (inf frames 640x480 at 30.00 FPS)

0: 480x640 Done. (1.238s)
0: 480x640 Done. (0.252s)
0: 480x640 Done. (0.193s)
0: 480x640 1 mate, Done. (0.631s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.171s)
0: 480x640 1 mate, Done. (0.176s)
0: 480x640 Done. (0.173s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.173s)
0: 480x640 Done. (0.175s)
0: 480x640 Done. (0.175s)
0: 480x640 1 

0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 Done. (0.172s)
0: 480x640 1 mate, Done. (0.177s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.177s)
0: 480x640 1 mate, Done. (0.177s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.177s)
0: 480x640 1 mate, Done. (0.176s)
0: 480x640 1 mate, Done. (0.176s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.174s)
0: 480x640 1 mate, Done. (0.177s)
0: 480x640 1 mate, Done. (0.181s)
0: 480x640 1 mate, Done. (0.172s)
0: 480x640 1 mate, Done. (0.

0: 480x640 Done. (0.180s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.175s)
0: 480x640 Done. (0.178s)
0: 480x640 Done. (0.178s)
0: 480x640 Done. (0.179s)
0: 480x640 Done. (0.180s)
0: 480x640 Done. (0.180s)
0: 480x640 Done. (0.178s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 D

0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.175s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 D

0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.175s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 1 mate, Done. (0.173s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.17

0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 D

0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 1 mate, Done. (0.175s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 Done. (0.177s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.175s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.176s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.175s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.174s)
0: 480x640 1 mate, Done. (0.173s)
0: 480x640 1 mate, Done.

0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.172s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.174s)
0: 480x640 Done. (0.173s)
0: 480x640 Done. (0.173s)
0: 480x640 D